In [1]:
pip install altair vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "vegafusion[embed]>=1.4.0"

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import altair as alt
from vega_datasets import data
import geopandas as gpd

In [5]:
import altair as alt
    
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [6]:
#Load excel data as pandas DataFrame
OECDData = pd.read_excel('OECD_betterLifeIndex.xlsx')

In [7]:
timeseries = pd.read_csv('time_series_data.csv')

In [8]:
import pandas as pd


filtered_df = timeseries[timeseries['Indicator'] == 'Long-term unemployment rate']


processed_timeseries = filtered_df[['Country', 'Time', 'Indicator', 'Value']]


processed_timeseries = filtered_df[['Country', 'Time', 'Value']].rename(columns={'Value': 'Long-term unemployment rate'})

processed_timeseries.to_csv('processed_dataset.csv', index=False)
processed_timeseries.head(10)

,Country,Time,Long-term unemployment rate
190,Australia,2004,0.968132
191,Australia,2005,0.905578
192,Australia,2006,0.881941
193,Australia,2007,0.824408
194,Australia,2008,0.806953
195,Australia,2009,1.167788
196,Australia,2010,1.019854
197,Australia,2011,1.004597
198,Australia,2012,1.047568
199,Australia,2013,1.150236


In [9]:
OECDData.head(10)

,Country,Dwellings without basic facilities,Housing expenditure,Rooms per person,Household net adjusted disposable income,Household net wealth,Labour market insecurity,Employment rate,Long-term unemployment rate,Personal earnings,...,Water quality,Stakeholder engagement for developing regulations,Voter turnout,Life expectancy,Self-reported health,Life satisfaction,Feeling safe walking alone at night,Homicide rate,Employees working very long hours,Time devoted to leisure and personal care
0,Australia,1.08,19.4,2.3,37433,528768,3.10,73,1.00,55206,...,92,2.7,92,83.0,85.0,7.1,67,0.9,12.5,14.36
1,Austria,0.80,20.8,1.6,37001,309637,2.30,72,1.30,53132,...,92,1.3,76,82.0,71.0,7.2,86,0.5,5.3,14.51
2,Belgium,0.70,20.0,2.1,34884,447607,2.40,65,2.30,54327,...,79,2.0,88,82.1,74.0,6.8,56,1.1,4.3,15.52
3,Canada,0.20,22.9,2.6,34421,478240,3.80,70,0.50,55342,...,90,2.9,68,82.1,89.0,7.0,78,1.2,3.3,14.57
4,Chile,9.40,18.4,1.9,18477,135787,7.00,56,4.15,26729,...,62,1.3,47,80.6,60.0,6.2,41,2.4,7.7,14.00
5,Colombia,12.30,20.5,1.0,13090,87431,9.25,58,1.10,13388,...,82,1.4,53,76.7,80.0,5.7,50,23.1,23.7,13.30
6,Costa Rica,2.30,17.0,1.2,16517,118723,9.27,55,1.50,18510,...,87,1.8,66,80.5,73.0,6.3,47,10.0,22.0,13.50
7,Czech Republic,0.50,23.4,1.5,26664,152114,2.30,74,0.60,29885,...,89,1.6,62,79.3,62.0,6.9,77,0.7,4.5,15.10
8,Denmark,0.50,23.3,1.9,33774,149864,4.50,74,0.90,58430,...,93,2.0,85,81.5,70.0,7.5,85,0.5,1.1,15.70
9,Estonia,5.70,17.0,1.7,23784,188627,5.40,74,1.20,30720,...,86,2.7,64,78.8,57.0,6.5,79,1.9,2.2,14.98


In [10]:
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
countries_shape = gpd.read_file(url) # zipped shapefile
countries_shape = countries_shape[['NAME', 'CONTINENT', 'ISO_A3', 'geometry']]


In [11]:
countries_shape.head(10)

,NAME,CONTINENT,ISO_A3,geometry
0,Fiji,Oceania,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,Tanzania,Africa,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,W. Sahara,Africa,ESH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,North America,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,North America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
5,Kazakhstan,Asia,KAZ,"POLYGON ((87.35997 49.21498, 86.59878 48.54918..."
6,Uzbekistan,Asia,UZB,"POLYGON ((55.96819 41.30864, 55.92892 44.99586..."
7,Papua New Guinea,Oceania,PNG,"MULTIPOLYGON (((141.00021 -2.60015, 142.73525 ..."
8,Indonesia,Asia,IDN,"MULTIPOLYGON (((141.00021 -2.60015, 141.01706 ..."
9,Argentina,South America,ARG,"MULTIPOLYGON (((-68.63401 -52.63637, -68.25000..."


In [12]:
catSelection = alt.selection_point(fields=['Country'])
#set up a conditional colouring based on selection
catColor = alt.condition(catSelection,
alt.Color('Country:N', legend=None), #hide the legend
alt.value('lightgray')
)

#set up a scatterplot with IMD Rank as y axis
#colour and shape by District Name
scatter = alt.Chart(OECDData).mark_point().encode(
y=alt.Y('Life satisfaction', scale=alt.Scale(domain=(4,8))),
color=catColor,
shape=alt.Shape('Country:N', legend=None)
).properties( #control the size property of the plot
width=150,
height=150
).transform_filter(
    catSelection
).interactive()



#Create a table layout of 7 plots, name the different plots for simplicity
incomePlot = scatter.encode(x=alt.X('Personal earnings', scale=alt.Scale(domain=(10000,70000))))
employmentPlot = scatter.encode(x=alt.X('Employment rate', scale=alt.Scale(domain=(35,80))))
educationPlot = scatter.encode(x=alt.X('Voter turnout', scale=alt.Scale(domain=(40,100))))
healthPlot = scatter.encode(x=alt.X('Self-reported health', scale=alt.Scale(domain=(25,90))))
crimePlot = scatter.encode(x=alt.X('Homicide rate', scale=alt.Scale(domain=(0,30))))
housingPlot = scatter.encode(x=alt.X('Rooms per person', scale=alt.Scale(domain=(1,3))))
livEnvPlot = scatter.encode(x=alt.X('Feeling safe walking alone at night', scale=alt.Scale(domain=(30,100))))
employeeHoursPlot = scatter.encode(x=alt.X('Employees working very long hours', scale=alt.Scale(domain=(0,30))))
                                                        
mcv = alt.vconcat(
alt.hconcat(incomePlot, employmentPlot, educationPlot, employeeHoursPlot),
alt.hconcat(healthPlot, crimePlot, housingPlot,livEnvPlot),
title='OECD Data Dashboard for Life Satisfaction Comparison'
).add_params(
catSelection
)

legend = alt.Chart(OECDData).mark_point().encode(
x=alt.X('Country'),
color=catColor,
shape = 'Country:N'
).add_params(
catSelection
)

chart1 = alt.vconcat(mcv, legend) 

In [13]:
options=['Dwellings without basic facilities', 'Housing expenditure',
       'Rooms per person', 'Household net adjusted disposable income',
       'Household net wealth', 'Labour market insecurity', 'Employment rate',
       'Long-term unemployment rate', 'Personal earnings',
       'Quality of support network', 'Educational attainment',
       'Student skills', 'Years in education', 'Air pollution',
       'Water quality', 'Stakeholder engagement for developing regulations',
       'Voter turnout', 'Life expectancy', 'Self-reported health',
       'Life satisfaction', 'Feeling safe walking alone at night',
       'Homicide rate', 'Employees working very long hours',
       'Time devoted to leisure and personal care']

x_dropdown_1 = alt.binding_select(options = options, name='Select a field for Bar Graph 1:')
x_selection_1 = alt.selection_point(fields=['X_feature_1'], bind=x_dropdown_1, name ='X_feature_1', value='Water quality')

catSelection = alt.selection_point(fields=['Country'])

catColor = alt.condition(catSelection,
alt.Color('Country:N', legend=None, scale=alt.Scale(scheme='paired')), #hide the legend
alt.value('lightgray')
)


bars1 = (
    alt.Chart(OECDData).mark_bar()
    .transform_fold(
        options, as_=['X_feature_1', 'Value']
    ).transform_filter(
        x_selection_1
    ).encode(
        x = alt.X('Value:Q', title = 'Chosen Indicator Value'),
        y = alt.Y('Country:N', title = 'Country', sort='-x'),
        color = catColor
).add_params(x_selection_1, catSelection)
).properties(
    title='Bar Chart 1'
)

In [14]:
options2=['Dwellings without basic facilities', 'Housing expenditure',
       'Rooms per person', 'Household net adjusted disposable income',
       'Household net wealth', 'Labour market insecurity', 'Employment rate',
       'Long-term unemployment rate', 'Personal earnings',
       'Quality of support network', 'Educational attainment',
       'Student skills', 'Years in education', 'Air pollution',
       'Water quality', 'Stakeholder engagement for developing regulations',
       'Voter turnout', 'Life expectancy', 'Self-reported health',
       'Life satisfaction', 'Feeling safe walking alone at night',
       'Homicide rate', 'Employees working very long hours',
       'Time devoted to leisure and personal care']

x_dropdown_2 = alt.binding_select(options = options2, name='Select a field for Bar Graph 2:')
x_selection_2 = alt.selection_point(fields=['X_feature_2'], bind=x_dropdown_2, name ='X_feature_2', value='Self-reported health')


bars2 = (
    alt.Chart(OECDData).mark_bar()
    .transform_fold(
        options2, as_=['X_feature_2', 'Value']
    ).transform_filter(
        x_selection_2
    ).encode(
        x = alt.X('Value:Q', title = 'Chosen Indicator Value'),
        y = alt.Y('Country:N', title = 'Country', sort='-x'),
        color = catColor
).add_params(x_selection_2, catSelection)
).properties(
    title='Bar Chart 2'
)

In [15]:
chart2 = (bars1 | bars2)

In [16]:
catSelection = alt.selection_point(fields = ['Country'])
catColor = alt.condition(catSelection, alt.Color('Country:N', legend=None, scale=alt.Scale(scheme='paired')), alt.value('lightgray'))
# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()
basemap = alt.layer(
alt.Chart(sphere).mark_geoshape(fill='white'),
alt.Chart(graticule).mark_geoshape(stroke='LightGray', strokeWidth=0.5)
)

time_series_chart = alt.Chart(processed_timeseries).mark_line(
    point=alt.OverlayMarkDef(filled=False, fill="white")
).encode(
    x='Time:T',
    y= 'Long-term unemployment rate:Q',
    color = catColor,
    shape = alt.Shape('Country:N', legend = None)
).transform_filter(catSelection).interactive()

average_data = processed_timeseries.groupby('Country')['Long-term unemployment rate'].mean().reset_index()


choropleth = alt.Chart(countries_shape).mark_geoshape().encode(
        color = alt.Color('Long-term unemployment rate:Q', scale = alt.Scale(scheme = 'inferno')),
        tooltip = [
        alt.Tooltip('NAME:N', title='Country'),
        alt.Tooltip('Long-term unemployment rate:Q', title='Long-term Unemployment Rate Average')
    ]).transform_lookup(
    lookup='NAME',
    from_=alt.LookupData(data=average_data, key='Country', fields=['Long-term unemployment rate'])
    )
countries = alt.Chart(countries_shape).mark_geoshape(fill='LightGray', stroke='white')

    

legend = alt.Chart(processed_timeseries).mark_point().encode(
    x=alt.X('Country'),
    color=catColor,
    shape = 'Country:N'
).add_params(catSelection)
chart3 = (((basemap + countries + choropleth).project('equalEarth').properties( width = 400, height = 400) | time_series_chart) & legend)

In [17]:
concatenated_chart = alt.hconcat(chart2, alt.vconcat(chart1, chart3))

In [18]:
concatenated_chart.save('multiple_charts.html')
